In [1]:
import randomVVdV
import sys
sys.path.append("/scratch/ratch/tjf324/pytorch-pretrained-BERT/")
from pytorch_pretrained_bert import modeling, tokenization
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
import logging
import torch
import numpy as np
import pandas as pd
import tqdm

from language_modeling.runners import (
    tokenize_example, InputExample,
    convert_example_to_features, features_to_data,
)

WNLI_TRAIN_PATH = "/scratch/tjf324/data/glue_auto_dl/WNLI/train.tsv"
WNLI_DEV_PATH = "/scratch/tjf324/data/glue_auto_dl/WNLI/dev.tsv"

import spacy
nlp = spacy.load('en_core_web_sm')

def get_pos(sent):
    return [token.pos_ for token in nlp(sent)]

def is_noun(pos):
    return pos in ["PRON", "PROPN", "NOUN"]

def get_pos_dict(sent):
    return {
        token.text.lower(): token.pos_
        for token in nlp(sent)
    }

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
device = torch.device("cuda:0")
bert_model_name = "bert-large-uncased"
max_sequence_length = 128
MASK = "[MASK]"

model = modeling.BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)
model.to(device)

tokenizer = tokenization.BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True);


param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=5e-5,
                     warmup=0.1,
                     t_total=300)

In [3]:
train_df = pd.read_csv(WNLI_TRAIN_PATH, sep="\t")
val_df = pd.read_csv(WNLI_DEV_PATH, sep="\t")

print((train_df["label"]==0).mean())
print((val_df["label"]==0).mean())

0.5086614173228347
0.5633802816901409


In [47]:
def remove_in_blacklist(chunk):
    BLACKLIST = ["he", "she", "it", "they", "who", "her", "we", "them",
                 "him", "his", "us", "me", "i", "anything", "anyone", "somebody", 
                 "someone"]
    return chunk.text.lower() not in BLACKLIST
    

def get_all_sentences(df, skip_false=False):
    sent1s, sent2s = [], []
    for _, row in tqdm.tqdm_notebook(df.iterrows(), total=len(df)):

        if skip_false:
            if not row.label:
                continue
        
        sent1, sent2 = row.sentence1, row.sentence2

        sent1s.append(sent1)
        sent2s_this = {"true": sent2, "alt": []}

        sent1_doc = nlp(sent1)
        sent2_doc = nlp(sent2)
        
        
        noun_chunks_sent1 = list(filter(remove_in_blacklist, sent1_doc.noun_chunks))
        noun_chunks_sent2 = list(filter(remove_in_blacklist, sent2_doc.noun_chunks))

        for chunk2 in noun_chunks_sent2[:1]:
            for chunk1 in noun_chunks_sent1:
          
                if chunk2.root.text.lower() == chunk1.root.text.lower():
                    continue
                alt_sent = sent2.replace(chunk2.text, chunk1.text)
                if alt_sent.lower() == sent2.lower():
                    continue
                sent2s_this["alt"].append(alt_sent)
        sent2s.append(sent2s_this)
        
    return sent1s, sent2s


In [8]:
# sent1s, sent2s = get_all_sentences(train_df, True)

In [50]:
preds = []
selected = []

stop_at = 1000

labels = train_df.label[:stop_at]
labels = [1] * len(sent1s)

model.train()

steps = 0

for sent1, sent2, label in tqdm.tqdm_notebook(zip(sent1s[:stop_at], 
                                                   sent2s[:stop_at], 
                                                   labels),
                                               total=min(stop_at, len(sent1s))):
    steps += 1
    example = InputExample(
        guid=0,
        text_a=sent1,
        text_b=sent2["true"],
        is_next=True,
    )
    tokenized_example = tokenize_example(example, tokenizer)
    tokenized_examples = [tokenized_example]
   
    for sent2_alt in sent2["alt"]:
        example = InputExample(
            guid=0,
            text_a=sent1,
            text_b=sent2_alt,
            is_next=True,
        )
        tokenized_example = tokenize_example(example, tokenizer)
        tokenized_examples.append(tokenized_example)
        
    if len(tokenized_examples) > 5:
        tokenized_examples = [tokenized_examples[0]] + list(np.random.choice(tokenized_examples, 2))
        

    
        
    features = [convert_example_to_features(ex, tokenizer, max_sequence_length, select_prob=0.0)
                for ex in tokenized_examples]
    
    batch = features_to_data(features).to(device)

    labels = torch.zeros(len(features), dtype=torch.long).to(device=device, )
    if label:
        labels[0] = 1
    
    loss = model(
                batch.input_ids, 
                batch.segment_ids, 
                batch.input_mask, 
                labels,
            )
    
    loss.backward()
    if not steps % 5:
        optimizer.step()
        optimizer.zero_grad()

            

In [51]:
model.eval()
preds = []
for sent1, sent2 in tqdm.tqdm_notebook(zip(val_sent1s, val_sent2s), total=len(val_sent1s)):
    example = InputExample(
        guid=0,
        text_a=sent1,
        text_b=sent2["true"],
        is_next=True,
    )
    tokenized_example = tokenize_example(example, tokenizer)
    features = [convert_example_to_features(tokenized_example, tokenizer, max_sequence_length, select_prob=0.0)]
    batch = features_to_data(features).to(device)
    with torch.no_grad():   
        result = model(
                batch.input_ids, 
                batch.segment_ids, 
                batch.input_mask, 
            )
    print(result)
    pred = int(torch.softmax(result, 1)[0, 1] > 0.3)
    preds.append(pred)

tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], device='cuda:0')
tensor([[0.8163, 0.0467]], devic